# Vessel ROI + Tiled Analysis Demo

This walkthrough configures the ROI used for processing, explains the tiled vessel extraction pipeline, and shows how to run an ROI scan across the stack.

## What you'll learn
- choose an ROI size/position and confirm what portion of the stack is analyzed
- tune the tiled processing profile (tile size, overlap, halo, blend mode, etc.)
- optionally sweep the ROI across the field using the built-in scanning helper

In [ ]:
%matplotlib inline
import os
import time
from pathlib import Path
from dataclasses import asdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches

plt.style.use('seaborn-v0_8-muted')

In [ ]:
PROJECT_ROOT = Path('..').resolve()
DATA_ROOT = PROJECT_ROOT / 'test' / 'test_files'
STACK_PATH = DATA_ROOT / '240207_002.czi'  # TODO: point to your actual stack
CONFIG_PATH = PROJECT_ROOT / 'config' / 'default_vessel_config.yaml'
OUTPUT_DIR = (PROJECT_ROOT / 'outputs' / 'roi_tiling_demo')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f'Project root: {PROJECT_ROOT}')
print(f'Data root   : {DATA_ROOT}')
print(f'Stack path  : {STACK_PATH}')
print(f'Config path : {CONFIG_PATH}')
print(f'Output dir  : {OUTPUT_DIR}')

if not STACK_PATH.exists():
    raise FileNotFoundError('Update STACK_PATH so it points at a real volume before running the pipeline.')

### Configure tiled analysis profiles
Pick one of the ready-made profiles or define your own tiling overrides. The values you set here drive tile generation, binarization, and blending.

In [ ]:
from VesselTracer.config import (
    load_stack_config,
    apply_stack_overrides,
    config_fingerprint,
    save_stack_config,
)

tiling_profiles = {
    'fast_preview': {
        'use_tiled_pipeline': True,
        'tile_xy': 256,
        'tile_overlap': 32,
        'halo_xy': 12,
        'vessel_scales': (0.8, 1.2, 1.8, 2.4),
        'anisotropic_z': True,
        'adaptive_closing': False,
        'closing_rmin': 1,
        'closing_rmax': 3,
        'layer_aware': True,
        'thresh_mode': 'triangle',
        'blend_mode': 'majority',
        'cache_intermediates': False,
    },
    'balanced': {
        'use_tiled_pipeline': True,
        'tile_xy': 384,
        'tile_overlap': 64,
        'halo_xy': 16,
        'vessel_scales': (1.0, 1.6, 2.3, 3.2, 4.5),
        'anisotropic_z': True,
        'adaptive_closing': True,
        'closing_rmin': 1,
        'closing_rmax': 5,
        'layer_aware': True,
        'thresh_mode': 'sauvola',
        'blend_mode': 'vesselness_max',
        'cache_intermediates': False,
    },
    'high_precision': {
        'use_tiled_pipeline': True,
        'tile_xy': 448,
        'tile_overlap': 128,
        'halo_xy': 24,
        'vessel_scales': (1.0, 1.4, 1.9, 2.6, 3.4, 4.4),
        'anisotropic_z': True,
        'adaptive_closing': True,
        'closing_rmin': 1,
        'closing_rmax': 6,
        'layer_aware': True,
        'thresh_mode': 'phansalkar',
        'blend_mode': 'or_then_clean',
        'cache_intermediates': True,
    },
}

profiles_table = pd.DataFrame(tiling_profiles).T
profiles_table

In [ ]:
SELECTED_PROFILE = 'balanced'  # <- change to any key in tiling_profiles
print(f'Using tiling profile: {SELECTED_PROFILE}')

base_stack_cfg = load_stack_config(CONFIG_PATH)
stack_overrides = {
    'notes': {'purpose': 'ROI + tiled demo', 'debug_level': 2},
    'tiling': tiling_profiles[SELECTED_PROFILE],
}
stack_cfg = apply_stack_overrides(base_stack_cfg, stack_overrides)
fingerprint = config_fingerprint(stack_cfg)
print(f'Active stack fingerprint: {fingerprint}')

pd.Series(asdict(stack_cfg.tiling))

### Instantiate the controller and set ROI parameters
Update `roi_overrides` to lock the ROI size or location for your dataset. Set `find_roi=False` if you prefer to process the full field.

In [ ]:
from VesselTracer.controller import VesselAnalysisController

roi_overrides = {
    'find_roi': True,
    'micron_roi': 520.0,
    'min_x': 1500,
    'min_y': 1200,
}

controller = VesselAnalysisController(
    input_path=STACK_PATH,
    config_path=CONFIG_PATH,
    verbose=stack_cfg.notes.get('debug_level', 1),
    use_gpu=False,
    stack_config=stack_cfg,
)

for key, value in roi_overrides.items():
    if value is not None:
        setattr(controller.config, key, value)

print(f"Controller ready (cfg={controller.stack_fingerprint[:8]})")
pd.Series({k: getattr(controller.config, k) for k in roi_overrides})

### Run the ROI pipeline (single ROI)
This call processes only the currently configured ROI. Use `run_multiscan` below if you want to sweep the ROI across the stack.

In [ ]:
%%time
controller.run_analysis(stack_config=stack_cfg)
roi = controller.roi_model
print(f'ROI volume shape: {roi.volume.shape}')

### Summarize intermediate volumes

In [ ]:
def summarize_volume(name, volume):
    if volume is None:
        return None
    arr = np.asarray(volume)
    return {
        'name': name,
        'shape': arr.shape,
        'dtype': str(arr.dtype),
        'min': float(arr.min()),
        'max': float(arr.max()),
        'mean': float(arr.mean()),
    }

summary_rows = [
    summarize_volume('Preprocessed ROI', getattr(roi, 'volume', None)),
    summarize_volume('Background', getattr(roi, 'background', None)),
    summarize_volume('Vesselness', getattr(roi, 'vesselness', None)),
    summarize_volume('Radius map', getattr(roi, 'radius', None)),
    summarize_volume('Binary mask', getattr(roi, 'binary', None)),
]
summary_rows = [row for row in summary_rows if row]
pd.DataFrame(summary_rows)

### Visualization helpers

In [ ]:
def _pick_z_indices(z_len, num_slices):
    return np.linspace(0, z_len - 1, num=min(num_slices, z_len), dtype=int)

def show_slice_grid(volume, title, num_slices=6, cmap='gray'):
    if volume is None:
        print(f'{title}: volume missing')
        return
    vol = np.asarray(volume)
    z_indices = _pick_z_indices(vol.shape[0], num_slices)
    fig, axes = plt.subplots(1, len(z_indices), figsize=(3 * len(z_indices), 3))
    for ax, z in zip(axes, z_indices):
        ax.imshow(vol[z], cmap=cmap)
        ax.set_title(f'z={z}')
        ax.axis('off')
    fig.suptitle(title)
    plt.show()

def show_mips(volume, title, cmap='inferno'):
    if volume is None:
        print(f'{title}: volume missing')
        return
    vol = np.asarray(volume)
    mip_xy = vol.max(axis=0)
    mip_xz = vol.max(axis=1)
    mip_yz = vol.max(axis=2)
    fig, axes = plt.subplots(1, 3, figsize=(14, 4))
    axes[0].imshow(mip_xy, cmap=cmap)
    axes[0].set_title('XY')
    axes[1].imshow(mip_xz, cmap=cmap)
    axes[1].set_title('XZ')
    axes[2].imshow(mip_yz, cmap=cmap)
    axes[2].set_title('YZ')
    for ax in axes:
        ax.axis('off')
    fig.suptitle(title)
    plt.show()

def plot_tile_overlay(volume, tiles, title='Tile layout (core footprints)'):
    vol = np.asarray(volume)
    mip = vol.max(axis=0)
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(mip, cmap='gray')
    for tile in tiles:
        (y0, y1), (x0, x1) = tile['core_bounds']
        rect = patches.Rectangle((x0, y0), x1 - x0, y1 - y0,
                                 linewidth=1.1, edgecolor='deepskyblue',
                                 facecolor='none', alpha=0.85)
        ax.add_patch(rect)
        ax.text(x0 + 4, y0 + 12, str(tile['id']), color='yellow', fontsize=7)
    ax.set_title(title)
    ax.axis('off')
    plt.show()

### Inspect the ROI volumes

In [ ]:
raw_roi = controller.image_model.create_roi(controller.config)
show_slice_grid(raw_roi.volume, 'Raw ROI slices')
show_slice_grid(roi.volume, 'Processed ROI slices', cmap='magma')
show_mips(roi.volume, 'Processed ROI MIPs')
show_slice_grid(getattr(roi, 'vesselness', None), 'Vesselness slices', cmap='plasma')
show_slice_grid(getattr(roi, 'binary', None), 'Binary mask slices', cmap='gray')

### Explore the tiled analysis outputs

In [ ]:
tile_cfg = controller.active_stack_config.tiling
tiles = controller.processor.generate_tiles(
    roi.volume,
    tile_xy=tile_cfg.tile_xy,
    overlap=tile_cfg.tile_overlap,
    halo_xy=tile_cfg.halo_xy,
)
print(f'Generated {len(tiles)} tiles (tile_xy={tile_cfg.tile_xy}, overlap={tile_cfg.tile_overlap}, halo={tile_cfg.halo_xy})')

tile_records = []
for tile in tiles:
    (y0, y1), (x0, x1) = tile['core_bounds']
    tile_records.append({
        'tile_id': tile['id'],
        'y0': y0,
        'y1': y1,
        'x0': x0,
        'x1': x1,
        'height': y1 - y0,
        'width': x1 - x0,
    })

tile_df = pd.DataFrame(tile_records)
tile_df.head()

In [ ]:
roi_shape = roi.volume.shape
roi_area = roi_shape[1] * roi_shape[2]
tile_core_area = (tile_df['height'] * tile_df['width']).sum()
redundancy = tile_core_area / roi_area
print(f'ROI XY: {roi_shape[2]} x {roi_shape[1]} pixels')
print(f'Nominal tile area (no overlap): {tile_cfg.tile_xy ** 2:,} px^2')
print(f'Total tile core area (with overlap): {tile_core_area:,} px^2')
print(f'Coverage redundancy factor: {redundancy:.2f}x')

In [ ]:
plot_tile_overlay(roi.volume, tiles)

### Helper: `run_multiscan()`
Wraps `controller.multiscan(...)` so there is a single obvious entry-point for multi-ROI scans.

In [ ]:
def run_multiscan(*, controller, skip_trace=False, **kwargs):
    print('Running multi-ROI scan (each tile uses the same tiled settings)...')
    return controller.multiscan(skip_trace=skip_trace, **kwargs)

### Optional: run `run_multiscan` across the stack
Enable `RUN_MULTISCAN` to march the ROI window across the XY plane. Each step calls `run_multiscan`, which in turn fans out to `controller.multiscan`.

In [ ]:
RUN_MULTISCAN = False  # Set to True to process every ROI position

if RUN_MULTISCAN:
    start = time.time()
    scan_results = run_multiscan(controller=controller, skip_trace=False)
    elapsed = time.time() - start
    print(f'Processed {len(scan_results)} ROI positions in {elapsed/60:.2f} min')
else:
    scan_results = []
    print('Set RUN_MULTISCAN = True to launch the scan (expect a long run).')

In [ ]:
if scan_results:
    scan_df = pd.DataFrame([
        {
            'center_x': entry['center_x'],
            'center_y': entry['center_y'],
            'min_x': entry['min_x'],
            'max_x': entry['max_x'],
            'min_y': entry['min_y'],
            'max_y': entry['max_y'],
            'paths_found': len(entry.get('paths', {})),
        }
        for entry in scan_results
    ])
    display(scan_df)
else:
    print('Scan skipped; nothing to summarize yet.')

### Save the active stack configuration for reproducibility

In [ ]:
cfg_path = OUTPUT_DIR / f'stack_cfg_{fingerprint}.json'
save_stack_config(stack_cfg, cfg_path)
print(f'Saved stack configuration to {cfg_path}')